# Train YOLOv11n for Golf Ball Detection

Fine-tunes YOLOv11 nano on the Sai Gon University golf ball dataset (~1,100 images).

**Instructions:**
1. Open in Google Colab
2. Runtime > Change runtime type > GPU (A100 if available, otherwise T4)
3. Run all cells
4. Download the trained model at the end

Training time: ~15-20 min on A100, ~45 min on T4

In [ ]:
# Install dependencies
!pip install roboflow ultralytics -q

In [ ]:
# Download dataset from Roboflow
from roboflow import Roboflow

rf = Roboflow(api_key="")
project = rf.workspace("sai-gon-university").project("golf-ball-and-hole-detection-1k7")
version = project.version(6)
dataset = version.download("yolov11")

print(f"\nDataset location: {dataset.location}")

In [ ]:
# Check dataset structure
import os

print("Dataset contents:")
!ls -la "{dataset.location}"
print("\ndata.yaml:")
!cat "{dataset.location}/data.yaml"

In [ ]:
from ultralytics import YOLO

# Load YOLOv11 nano
model = YOLO('yolo11n.pt')
print("Loaded YOLOv11n")

In [ ]:
# Train the model
results = model.train(
    data=f"{dataset.location}/data.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    patience=20,
    device=0,
    workers=2,
    project='golf_ball_detector',
    name='yolo11n_golfball',
    pretrained=True,
    optimizer='AdamW',
    lr0=0.001,
    augment=True,
)

In [ ]:
# Evaluate
metrics = model.val()

print("\n" + "="*40)
print("FINAL METRICS")
print("="*40)
print(f"mAP50:     {metrics.box.map50:.3f}")
print(f"mAP50-95:  {metrics.box.map:.3f}")
print(f"Precision: {metrics.box.mp:.3f}")
print(f"Recall:    {metrics.box.mr:.3f}")
print("="*40)

In [ ]:
# Export to NCNN for Raspberry Pi
best_model = YOLO('golf_ball_detector/yolo11n_golfball/weights/best.pt')
best_model.export(format='ncnn', imgsz=640)
print("Exported to NCNN format!")

In [ ]:
# Download models
from google.colab import files

!cp golf_ball_detector/yolo11n_golfball/weights/best.pt golf_ball_yolo11n.pt
print("Downloading PyTorch model...")
files.download('golf_ball_yolo11n.pt')

!zip -r -q golf_ball_yolo11n_ncnn.zip golf_ball_detector/yolo11n_golfball/weights/best_ncnn_model/
print("Downloading NCNN model...")
files.download('golf_ball_yolo11n_ncnn.zip')

print("\nDone!")

## Usage on Raspberry Pi

```bash
# Copy to Pi
scp golf_ball_yolo11n.pt pi@<ip>:~/openlaunch/models/
scp golf_ball_yolo11n_ncnn.zip pi@<ip>:~/openlaunch/models/

# On Pi
cd ~/openlaunch/models && unzip golf_ball_yolo11n_ncnn.zip

# Test
cd ~/openlaunch
python scripts/test_yolo_detection.py \
  --model models/golf_ball_yolo11n.pt \
  --confidence 0.3 --max-det 1 \
  --width 1280 --height 720 \
  --headless --num-frames 10
```